In [4]:
import numpy as np
import cmath
import math
import matplotlib.pyplot as plt 
import scipy.integrate as integrate

In [99]:
n=1.32
d=.3

wavelength = np.arange(start=1, stop=10, step=0.1).reshape(90,1)

delta = ((2*cmath.pi)/wavelength)*n*d
exp_plus = np.exp(1j*delta)
exp_minus = np.exp(-1j*delta)

zero = np.ones(wavelength.shape)

t=np.ravel(np.array([exp_plus,zero,zero,exp_minus]), order="F").reshape(90,2,2)

polar = np.vectorize(cmath.polar)

T_1_1 = polar(t.ravel()[::4])[0]
T_2_1 = polar(t.ravel()[2::4])[0]
polar(t)[0]

array([[[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.]],

       [[1.,

In [94]:
np.divide(T_2_1,T_1_1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])